In [1]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# Notebook for data analysis of all things HubCoin

# Sections:
# 1. HubCoinMonitor: coordinates wallets with the manual addressbook from the hubcoin channel on GHub Discord.
#        Uses: Balance inquiries, basic auditing
#        Files: CLEAN addressbook.csv from the google sheet locate here:
#                   https://docs.google.com/spreadsheets/d/1m1aJ9We3B-Hhjn3a0JSlR-_VRMWC6_gHf5iYIvRqgh0
#               and the csv from this page:
#                   https://goerli.etherscan.io/token/0xaaeddee8a8112ffdb798a02ca19a732e1434b6e6#balances
# 2. TreasuryMonitor: tracks the flow of HubCoin from the GH Issuing Authority wallet address, ie. the treasury
#        Uses: Tracking distribution vs. government spending, tracking 







# HubCoinTreasuryMonitor (gonna revert the name to monitor soon) jupyter notebook

# First we copy almost all of HCM
# :HCM:
# Holders is our manually created record of addresses and names
# Import **** <<<<< THIS FILENAME SHOULDN'T NEED CHANGING BUT DOUBLE-CHECK >>>>>  
Holders=pd.read_csv('HubCoin Address Book & Achievement List - Address Book.csv', thousands=',')
# Make sure the hex addresses are lowercase (for some reason they aren't universal in case -_-)
Holders['ADDRESSES'] = Holders['ADDRESSES'].str.lower()
# Line up the addresses (this may be redundant as I iterate through to find the right address...may change later)
Holders = Holders.sort_values(by='ADDRESSES')
# Useless Column
Holders.drop('Achievements',axis=1,inplace=True)
# Take a loook-seeeee:
#Holders.head(5)


# AddressBook is actually our downloaded csv from etherscan containing balances and hex addresses
# Import 2 ***** <<<<< THIS FILENAME ALMOST CERTAINLY NEEDS CHANGING >>>>
Records = pd.read_csv('tokenholders32022.csv')
Records['HolderAddress'] = Records['HolderAddress'].str.lower()
Records.drop('PendingBalanceUpdate',axis=1,inplace=True)
#ESRecords.head(5)


# Let's just make everything into lists LOL
# We are creating two dictionaries: one of name:address pairs, and one of address:balance pairs
# List of names
namlist = Holders['Player Name'].tolist()
addlist = Holders['ADDRESSES'].tolist()
logBook = {}
logBook = dict(zip(namlist, addlist))
invlogBook = {v: k for k, v in logBook.items()}
# Same thing but on the other side with a one liner because fuck you
addressBalance = list(zip(Records.HolderAddress, Records.Balance.apply(str)))


# Create a dictionary to put our name:balance pairs into
nameBalance = {}

# The following function prints the name, balance, and hex address of all accounts, and populates the nameBalance dict
for key,value in addressBalance:
        for x in logBook:
            if key == logBook[x]:
                
                nameBalance[x]= value
            
                # Uncomment the following lines to printout all info in this cell
                #print(x)
                #print("BALANCE:    " + value + '\r\nADDRESS:    ' + key + '\r\n')
            
            else:
                pass


# create and populate a reverse dict (fuk u I like dictionaries)
invNameBalance = {}
invNameBalance = {v: k for k, v in nameBalance.items()}


#******************************************************************************

# OKAY NOW FOR THE TREASURY EXPENSES MONITOR

# Step one: import the csvs for each month (actually fuck that let's rip the full one)

# This expense report is essentially showing us every new mint

TreasuryExpenses=pd.read_csv('TreasuryExpensesUpToApril12022.csv', thousands=',')

# OKAY so we DO have to convert it first because its imported as a string
TreasuryExpenses["DateTime"]=TreasuryExpenses["DateTime"].apply(lambda x : pd.Timestamp(x))

# Just seems easier to Have columns with this stuff by itself, idk
TreasuryExpenses["Month"] = TreasuryExpenses["DateTime"].apply(lambda x : x.month)
TreasuryExpenses["Year"] = TreasuryExpenses["DateTime"].apply(lambda x : x.year)

# Clean dataframes with the garbage chopped off
TreasuryExpenses.drop(['Txhash', 'Blockno'], axis=1, inplace=True)
TESlim = TreasuryExpenses.drop(['Method', 'UnixTimestamp'],axis=1)

# We don't care about other transactions, just what's coming out of the vault
TESlim = TESlim[TESlim['From']=='0x196bd46fe0b188d699ed882ff1e2df11dcb03067']
#TreasuryExpenses.head(4)
#TESlim.head(4)



GHAddresses = []
for key in logBook:
    if "GH" in key:
        GHAddresses.append(logBook[key])

# Hey dumbass you forgot about the treasury itself having a GH prefix lmao
GHAddresses.remove('0x196bd46fe0b188d699ed882ff1e2df11dcb03067')



# Buuuut maybe we wanna save the total expenditures before we drop down to only distro
TotalMonthlyExpenditures = TESlim.groupby('Month').sum()

TEStwo = TESlim.copy()

TotalMonthlyExpenditures = TESlim.groupby('Month').sum()

TEStwo = TESlim.copy()

# Now we should be able to pass this list of gv addressses to the dataframe
# The question is, how do we want to remove these from the balance calculation?
# We can iterate through the addresses and set their balance to zero before doing the sum() line...
# if we put this all into a single function then we won't have to worry about inplace or any of that
# buuuiiiiilllll shite

for x in GHAddresses:
        
        # for each entry in the "To" column..
    for y in TESlim["To"]:
            
            # if they are the same..
        if x == y:
                
                # Zero that row's Quantity
            
            TEStwo.loc[TEStwo['To'] == x, 'Quantity'] = 0
                
        else:
            pass
        
        
TotalMonthlyDistribution = TEStwo.groupby('Month').sum()


#**************************************************************************

# TIME TO LOOK AT ALL TRANSACTIONS

In [2]:
# ALright, so first we need to import the csv with a record of everything up until the date it's requested
# This can be found at: 
#     https://goerli.etherscan.io/token/0xaaeddee8a8112ffdb798a02ca19a732e1434b6e6?a=0x196bd46fe0b188d699ed882ff1e2df11dcb03067

AllTransactions = pd.read_csv("TransactionsFirstHalf.csv",thousands=',')

# And then let's drop the stuff we don't care about
AllTransactions.drop(['Blockno'],axis=1,inplace=True)

# If we want to see how the economy itself is doing, we need to remove all the nose coming from the treasury









In [3]:
AllTransactions.head()

,Txhash,UnixTimestamp,DateTime,From,To,Quantity,Method
0,0x3e75d4848492f345aad602043028a0e211860f2282cd...,1635626112,2021-10-30 20:35:12,0x0000000000000000000000000000000000000000,0x196bd46fe0b188d699ed882ff1e2df11dcb03067,16161616.0,-
1,0x5c3e46bba9d3cf5a5ee5d4e913ec0a5c5d6b36595542...,1635628409,2021-10-30 21:13:29,0x196bd46fe0b188d699ed882ff1e2df11dcb03067,0xcbafc87ef7cbdb7d818a23350daa5370b6b5fba1,1.0,Transfer
2,0xc8858f6186eabe2bac1aa96ce63dbed69ef0407eec22...,1635629024,2021-10-30 21:23:44,0x196bd46fe0b188d699ed882ff1e2df11dcb03067,0xcbafc87ef7cbdb7d818a23350daa5370b6b5fba1,4.0,Transfer
3,0xcdf468949c16c8b5416fdc6b1cb7dbe6021e5ac528ad...,1635634127,2021-10-30 22:48:47,0x196bd46fe0b188d699ed882ff1e2df11dcb03067,0x9a3bc9ff044f12b92349f9c0e6bae5da8af731c5,5.0,Transfer
4,0xdc0fea5c40726cef34d716bf386b01cda0e2e8507f20...,1635815878,2021-11-02 01:17:58,0x196bd46fe0b188d699ed882ff1e2df11dcb03067,0x6fd2e7d2c9e0f84f58c08ffb8ef396836e47d96c,400.0,Multisend Token


In [4]:
# We can measure the activity of all accounts
AllTransactions.groupby("From").nunique()

,Txhash,UnixTimestamp,DateTime,To,Quantity,Method
From,,,,,,
0x0000000000000000000000000000000000000000,1,1,1,1,1,1
0x06a84a718c0fc3812cb308a84e130ab383dbbe14,1,1,1,1,1,1
0x196bd46fe0b188d699ed882ff1e2df11dcb03067,116,115,115,84,92,3
0x47070a5738a3c2b59b6e6de4550ac3614a10e9c7,1,1,1,1,1,1
0x4fe3e8f76636b677f1e203eb0c0a3733e6ec6020,1,1,1,1,1,1
0x57cc626a0de7de3f4ec831c49f2a8a70720c0b8a,1,1,1,1,1,1
0x6fd2e7d2c9e0f84f58c08ffb8ef396836e47d96c,2,2,2,1,2,1
0x7fba3fd5bf95a5a1c4ce7b691f64d94ea89287fd,56,56,56,14,17,1
0x8391c49c43cc0941c7e1b647ed706527d5b792da,7,7,7,5,2,1


In [8]:
UniqueRecipiants = AllTransactions.groupby("To").nunique()

In [11]:
UniqueRecipiants.to_csv('ur.csv')

In [ ]:
# INSIGHTS

In [ ]:
#TESlim.loc[TESlim['Quantity'] >= 10000]
#TreasuryExpenses.loc[TreasuryExpenses['Quantity'] >5000]
#TreasuryExpenses[TreasuryExpenses['Quantity']==100]
#TEStwo[(TEStwo['Quantity']==150)].To.nunique()



In [ ]:
TreasuryExpenses.plot.scatter(x='DateTime',y='Quantity',figsize=(15,3))

In [ ]:
TESlim.plot.scatter(x='DateTime',y='Quantity',figsize=(15,10))

In [ ]:
TotalMonthlyExpenditures.Quantity.sum()

In [ ]:
TotalMonthlyDistribution.Quantity.sum()

In [ ]:
(TotalMonthlyExpenditures - TotalMonthlyDistribution).Quantity.sum()